<a href="https://colab.research.google.com/github/shebbar27/bert-test/blob/main/BertTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 4.5 MB/s 
     |████████████████████████████████| 77 kB 3.8 MB/s 
     |████████████████████████████████| 6.5 MB 52.2 MB/s 
     |████████████████████████████████| 895 kB 53.5 MB/s 
     |████████████████████████████████| 596 kB 43.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
input_from_train_ds = [['Raise the ruby cup'],
                       ['Fill a little into the cobalt basin'],
                       ['Elevate the indigo grail'],
                       ['Fill a little into the meager rectangular cardinal basin']];

input_from_unseen_ds_similar = [['Choose the ruby cup'],
                                ['Fill the cobalt basin slightly'],
                                ['The indigo grail should be lifted'],
                                ['Unload a little into the meager rectangular cardinal basin']];
"""
input_from_unseen_ds_different = [['Drop the blue cup'],
                                  ['Throw the red basin far away'],
                                  ['The green box should be thrown away'],
                                  ['Fill a lot into the circular bowl']];
"""

# """
input_from_unseen_ds_different = [['This is a totally unrelated sentance'],
                                  ['This statement makes no sense'],
                                  ['At least this sentance should have a small similarity sore'],
                                  ['It doesn\'t seem like a good similarity measure']];
# """

"""
input_from_unseen_ds_different = [['asd weweg asegeg erherherh ssadff'],
                                  ['tu sdfhsdh rwey fghk hfgkdtghk dfsg'],
                                  ['nc hfg waet dfnhd awrawwf hfgjdfg yktg khjla asd dfhd lyh'],
                                  ['tj dfgzdf dfhdh fcd cbdbd rsejj dfhsgdsaghd']];
"""

"\ninput_from_unseen_ds_different = [['asd weweg asegeg erherherh ssadff'],\n                                  ['tu sdfhsdh rwey fghk hfgkdtghk dfsg'],\n                                  ['nc hfg waet dfnhd awrawwf hfgjdfg yktg khjla asd dfhd lyh'],\n                                  ['tj dfgzdf dfhdh fcd cbdbd rsejj dfhsgdsaghd']];\n"

In [3]:
import numpy as np
from numpy.linalg import norm
from tensorflow.python.ops.numpy_ops import np_config
from transformers import BertTokenizer, TFBertModel

np_config.enable_numpy_behavior()

LANGUAGE_TOKEN_MAX_LENGTH = 15

bertModel = TFBertModel.from_pretrained("bert-base-cased", 
                                      output_hidden_states = True)
bertModel.summary()

def tokenizeForBert(language):
  tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
  bertTokens = tokenizer(
      language, 
      return_tensors='np',
      return_attention_mask=False, 
      return_token_type_ids=False,
      max_length=LANGUAGE_TOKEN_MAX_LENGTH,
      padding='max_length')
  return bertTokens

def getBertOutput(tokens): 
  output = bertModel(tokens)
  return output['pooler_output']

def getBertEmbeddings(languageInputs):
  bertEmbeddings = []
  for language in languageInputs:
    tokens = tokenizeForBert(language)
    bertOutput = getBertOutput(tokens)
    bertEmbeddings.append(bertOutput)
  return bertEmbeddings

def distanceBetweenVectors(vector1, vector2):
  return np.linalg.norm(vector1 - vector2)

def distanceBetweenBertEmbeddings(bertEmbeddings1, bertEmbeddings2):
  sentenceDistances = []
  for (array2d_1, array2d_2) in zip(bertEmbeddings1, bertEmbeddings2):
    distance = distanceBetweenVectors(array2d_1, array2d_2)
    sentenceDistances.append(distance)
  return sentenceDistances

def cosineSimilarityBetweenBertEmbeddings(bertEmbeddings1, bertEmbeddings2):
  sentenceSimilarity = []
  for (array2d_1, array2d_2) in zip(bertEmbeddings1, bertEmbeddings2):
    # array2d_1, array2d_2 are of dimensions 1x768 
    cosineSimilarity = np.dot(array2d_1[0], array2d_2[0])/(norm(array2d_1[0]) * norm(array2d_2[0]))
    sentenceSimilarity.append(cosineSimilarity)
  return sentenceSimilarity

def getTokensAndBertOutputFromSampleSentence(language):
  tokens = tokenizeForBert(language)
  output = getBertOutput(tokens)
  return tokens, output

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/502M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "tf_bert_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  108310272 
                                                                 
Total params: 108,310,272
Trainable params: 108,310,272
Non-trainable params: 0
_________________________________________________________________


In [4]:
tokens1, output1 = getTokensAndBertOutputFromSampleSentence(input_from_train_ds[0])
tokens2, output2 = getTokensAndBertOutputFromSampleSentence(input_from_unseen_ds_different[0])
print(tokens1)
print(tokens2)
print()

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

{'input_ids': array([[  101, 20089,  2217,  1103, 16259,  1183,  4355,   102,     0,
            0,     0,     0,     0,     0,     0]])}
{'input_ids': array([[  101,  1188,  1110,   170,  5733, 16037,  1850,  3923,   102,
            0,     0,     0,     0,     0,     0]])}



In [5]:
output_of_train_ds = getBertEmbeddings(input_from_train_ds)
output_of_unseen_ds_similar = getBertEmbeddings(input_from_unseen_ds_similar)
output_of_unseen_ds_different = getBertEmbeddings(input_from_unseen_ds_different)

# embeddingDistancesSimilar = distanceBetweenBertEmbeddings(output_of_train_ds, output_of_unseen_ds_similar)
cosineSimilarity = cosineSimilarityBetweenBertEmbeddings(output_of_train_ds, output_of_unseen_ds_similar)
print("Bert Embedding distances for similar language inputs: ")
for i in range(len(input_from_train_ds)):
  print(f"Trained Input: {input_from_train_ds[i][0]}")
  print(f"Unseen Input Similar: {input_from_unseen_ds_similar[i][0]}")
  # print(f"Vector distance between Bert Embeddings: {embeddingDistancesSimilar[i]}")
  print(f"Pairwise cosine similarity between Bert Embeddings: {cosineSimilarity[i]}")
  print()

# embeddingDistancesDifferent = distanceBetweenBertEmbeddings(output_of_train_ds, output_of_unseen_ds_different)
cosineSimilarity = cosineSimilarityBetweenBertEmbeddings(output_of_train_ds, output_of_unseen_ds_different)
print("Bert Embedding distances for different language inputs: ")
for i in range(len(input_from_train_ds)):
  print(f"Trained Input: {input_from_train_ds[i][0]}")
  print(f"Unseen Input Different: {input_from_unseen_ds_different[i][0]}")
  # print(f"Vector distance between Bert Embeddings: {embeddingDistancesDifferent[i]}")
  print(f"Pairwise cosine similarity between Bert Embeddings: {cosineSimilarity[i]}")
  print()

Bert Embedding distances for similar language inputs: 
Trained Input: Raise the ruby cup
Unseen Input Similar: Choose the ruby cup
Pairwise cosine similarity between Bert Embeddings: 0.9981473684310913

Trained Input: Fill a little into the cobalt basin
Unseen Input Similar: Fill the cobalt basin slightly
Pairwise cosine similarity between Bert Embeddings: 0.9897114634513855

Trained Input: Elevate the indigo grail
Unseen Input Similar: The indigo grail should be lifted
Pairwise cosine similarity between Bert Embeddings: 0.9940943121910095

Trained Input: Fill a little into the meager rectangular cardinal basin
Unseen Input Similar: Unload a little into the meager rectangular cardinal basin
Pairwise cosine similarity between Bert Embeddings: 0.9989982843399048

Bert Embedding distances for different language inputs: 
Trained Input: Raise the ruby cup
Unseen Input Different: This is a totally unrelated sentance
Pairwise cosine similarity between Bert Embeddings: 0.9947072863578796

Trai